In [62]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Add, BatchNormalization, Activation
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import tensorflow.keras.backend as K

In [63]:
class Dataset(tf.keras.utils.Sequence):
    def __init__(self, dataset="train", batch_size=64):
        df = pd.read_csv("dogs.csv")
        df = df.loc[df['data set'] == dataset]
        self.file_paths = np.array(df["filepaths"])
        self.labels = np.array(df["labels"])
        ordering = np.random.permutation(range(len(self.file_paths)))
        self.file_paths = self.file_paths[ordering]
        self.labels = self.labels[ordering]
        self.a = self.labels
        encoder = OneHotEncoder(sparse=False)
        labels_2d = [[x] for x in self.labels]
        self.labels = encoder.fit(labels_2d).transform(labels_2d)
        self.batch_size = batch_size
    
    def __getitem__(self, batch_index):
        start_index = batch_index * self.batch_size
        end_index = (batch_index+1) * self.batch_size
        out = np.array([np.array(Image.open(x)) for x in self.file_paths[start_index:end_index]]) # pre shuffled
        out = (out / (255/2)) - 1
        labels = tf.stack(self.labels[start_index:end_index])
        return out, np.array(labels)
    
    def __len__(self):
        return len(self.labels) // self.batch_size

In [64]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###

    return X

In [65]:
def convolutional_block(X, f, filters, stage, block, s=2):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    ### END CODE HERE ###
    return X

In [43]:
train = Dataset("train")
test = Dataset("test")
valid = Dataset("valid")

kernel_size = 3

model = Sequential ([
    tf.keras.layers.Input((244,244,3)),
    Conv2D(64, kernel_size),
    tf.keras.layers.ReLU(),
    MaxPooling2D(),
    Conv2D(32, kernel_size),
    tf.keras.layers.ReLU(),
    MaxPooling2D(),
    Conv2D(16, kernel_size),
    tf.keras.layers.ReLU(),
    MaxPooling2D(),
    Conv2D(8, kernel_size),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalMaxPooling2D(),
    Dense(70, activation='softmax'),
])


model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
K.set_value(model.optimizer.learning_rate, 0.001)

model.summary()
model.fit(
  train,
  epochs=3,
  validation_data=test,
)





Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_54 (Conv2D)          (None, 242, 242, 64)      1792      
                                                                 
 re_lu_53 (ReLU)             (None, 242, 242, 64)      0         
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 121, 121, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_55 (Conv2D)          (None, 119, 119, 32)      18464     
                                                                 
 re_lu_54 (ReLU)             (None, 119, 119, 32)      0         
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 59, 59, 32)       0         
 g2D)                                                

In [66]:
train = Dataset("train")
test = Dataset("test")
valid = Dataset("valid")

kernel_size = 3

def get_model():
    inputs = Input((244, 244, 3))
    x = Conv2D(64, 7, 2)(inputs)
    x = BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = MaxPooling2D(3)(x)
    x = convolutional_block(x, 3, [64,64,256], 1, "a")
    x = identity_block(x, 3, [64,64,256], 1, "b")
    x = identity_block(x, 3, [64,64,256], 1, "c")
    x = convolutional_block(x, 3, [128,128,512], 2, "a")
    x = identity_block(x, 3, [128,128,512], 2, "b")
    x = identity_block(x, 3, [128,128,512], 2, "c")
    x = identity_block(x, 3, [128,128,512], 2, "d")
    x = convolutional_block(x, 3, [256, 256, 1024], 2, "a")
    x = identity_block(x, 3, [256, 256, 1024], 2, "b")
    x = identity_block(x, 3, [256, 256, 1024], 2, "c")
    x = identity_block(x, 3, [256, 256, 1024], 2, "d")
    x = identity_block(x, 3, [256, 256, 1024], 2, "e")
    x = identity_block(x, 3, [256, 256, 1024], 2, "f")
    x = convolutional_block(x, 3, [512, 512, 2048], 2, "a")
    x = identity_block(x, 3, [256, 256, 2048], 2, "b")
    x = identity_block(x, 3, [256, 256, 2048], 2, "c")
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    x = Dense(70, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=x, name="model")
    return model

model = get_model()
    
model.compile(
    'adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)
    
    

K.set_value(model.optimizer.learning_rate, 0.001)

model.summary()
model.fit(
  train,
  epochs=3,
  validation_data=test,
)


ValueError: The name "res2a_branch2a" is used 3 times in the model. All layer names should be unique.